In [14]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [23]:
df = pd.read_csv("data/cf_train_no_noise.csv")
df_noise = pd.read_csv("data/cf_train.csv")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [24]:
def encode(value, encoding):
    for key, val in encoding.items():
        if value == key:
            return val

class_values_era = list(df.era.unique())
class_values_era.sort()
class_values_target = list(df.target_10_val.unique())
class_values_target.sort()
era_encoding = {val: i for i, val in enumerate(class_values_era)}
target_encoding = {val: i for i, val in enumerate(class_values_target)}
df["era"] = df["era"].apply(encode, args=(era_encoding,))
df["target_5_val"] = df["target_5_val"].apply(encode, args=(target_encoding,))
df["target_10_val"] = df["target_10_val"].apply(encode, args=(target_encoding,))

df_noise["era"] = df_noise["era"].apply(encode, args=(era_encoding,))
df_noise["target_5_val"] = df_noise["target_5_val"].apply(encode, args=(target_encoding,))
df_noise["target_10_val"] = df_noise["target_10_val"].apply(encode, args=(target_encoding,))

dataset = df
target_column = "target_10_val"
output_classes = 5
shuffle = False

In [25]:
class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.X = self.df.iloc[:, :-7]
        self.y = self.df[target_column]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        return (
            torch.tensor(self.X.iloc[index].values, dtype=torch.float32, device=device),
            torch.tensor(self.y.iloc[index].values, dtype=torch.long, device=device),
        )

In [26]:
if shuffle:
    dataset = dataset.sample(frac=1).reset_index(drop=True)

train_size = int(0.85 * len(dataset))
train_dataset = CustomDataset(dataset[:train_size])
val_dataset = CustomDataset(dataset[train_size:])

train_loader = DataLoader(train_dataset, batch_size=64)
val_loader = DataLoader(val_dataset, batch_size=64)

In [34]:
class MLP(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        self.layer_sizes = layer_sizes
        self.layers = nn.ModuleList()
        self.softmax = nn.Softmax(dim=1)
        
        for i in range(1, len(self.layer_sizes)):
            if i == len(self.layer_sizes) - 1:
                self.layers.append(nn.Linear(self.layer_sizes[i - 1], self.layer_sizes[i]))
            else:
                self.layers.append(nn.Linear(self.layer_sizes[i - 1], self.layer_sizes[i]))
                self.layers.append(nn.ReLU())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.softmax(x)
        return x

In [35]:
# set layer sizes here
layer_sizes = []
model = MLP(layer_sizes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(model, criterion, optimizer, train_loader, val_loader, epochs=10):

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in tqdm(range(epochs)):
        model.train()
        train_loss = 0
        train_accuracy = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            train_accuracy += (predicted == labels).sum().item()

        train_loss /= len(train_loader)
        train_accuracy /= len(train_loader.dataset)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        model.eval()
        val_loss = 0
        val_accuracy = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                val_accuracy += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch + 1}/{epochs}: "
                f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
                f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")